In [42]:
from bs4 import BeautifulSoup
from dateutil.parser import parse
from datetime import datetime
from datetime import datetime, timedelta
import urlparse
import pymongo 
from pymongo import MongoClient
import os

In [43]:
# def get_draft_date(soup):
#     draft_deets = soup.find("div", {"class": "games-alert-mod alert-mod2 games-grey-alert"})
#     draft_date = None
#     for b in draft_deets.findAll('b'):
#         key = b.text.replace(':', '').strip()
#         val = b.nextSibling.strip()
#         if key=='Draft Date':
#             val = b.nextSibling.replace(',','').replace('.','')[4:].strip()
#             draft_date = datetime.strptime(val, '%b %d %Y')
#     return draft_date

In [44]:
def get_league_draft_picks(soup):
   
    connection = MongoClient()
    db = connection['espn_leagues_drafts']
    auction_drafts = db['auction_drafts']
    
    #draft_date = get_draft_date(soup)
    draft = soup.find("div", {"class": "games-fullcol games-fullcol-extramargin"})
    draft_results=draft.find('table')
    league_picks = []
    for team in draft_results.findAll('table'):
        team_name = team.find('a').text
        team_link = team.find('a')['href']
        parsed=urlparse.urlparse(team_link)
        team_id=urlparse.parse_qs(parsed.query)['teamId'][0]
        season_id = urlparse.parse_qs(parsed.query)['seasonId'][0]
        league_id = urlparse.parse_qs(parsed.query)['leagueId'][0]

        #skip the first row because it has team info
        for pick_details in team.findAll('tr')[1:]:

            for i, pick_detail in enumerate(pick_details.findAll('td')):
                if i%3==0:
                    nom_order=int(pick_detail.text)
                elif (i-1)%3==0:
                    player_link = pick_detail.find('a')
                    if not player_link == None:
                        player_name = player_link.text
                        player_id = player_link['playerid']
                        #not sure what this is yet....
                        unique_team_id = player_link['teamid']
                    else:
                        player_name = ''
                        player_id = -9999999999
                        unique_team_id = -9999999999
                else:
                    price = int(pick_detail.text.replace('$',''))

            pick_dict = {"league_id":league_id,
                         #"draft_date":draft_date,
                        "team_name":team_name,
                        "team_id":team_id,
                        "unique_team_id":unique_team_id,
                        "season_id":season_id,
                        "nom_order":nom_order,
                        "player_name":player_name,
                        "player_id":player_id,
                        "player_name_id":"%s (%s)"%(player_name, player_id),
                        "price":price}
            
            auction_drafts.insert_one(pick_dict)
            #league_picks.append(pick_dict)
            
    return league_picks

In [45]:
def get_all_picks(year):

    #all_picks = []
    for draft_recap in os.listdir(year):
        #print draft_recap
        if not draft_recap.startswith("."):
            draft_file = open('%s/%s'%(year,draft_recap),'r')
            soup = BeautifulSoup(draft_file,"lxml")
            #print draft_file
            get_league_draft_picks(soup)
            draft_file.close()
            
    return None

In [46]:
# connection = MongoClient()
# db = connection['espn_leagues_drafts']
# drafts = db['drafts']
# drafts.drop()
# drafts = db['drafts']
# drafts.insert_many()

In [47]:
get_all_picks('2015')